In [2]:
# Importing libraries
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight') 

import math

from datetime import datetime, timedelta

from yahoofinancials import YahooFinancials
from joblib import Memory

In [4]:
# Create a list of stocks from the directory. These will be used to while creating a dataframe
list_of_stocks = os.listdir("Data/Stocks")

In [194]:
# Create Stock Dataframe. Take the first n stocks from the list and create a dataframe from them
def create_stock_dataframe(n):
    stocks = pd.DataFrame({'Ticker' : []})
    
    for i in range(0, n):
        # Read through the documents, reformatting the datatime so it can be used with pandas
        data = pd.read_csv('Data/Stocks/' + list_of_stocks[i])
        data['Date'] = pd.to_datetime(data['Date'])
        data = data.set_index('Date')
        ticker =  list_of_stocks[i].split('.')[0]
        # Drop irrelevent columns and select desired range
        # here I am selecting January and February from 2016
        data = data['2016-01':'2016-2'].drop(['Open', 'High', 'Low', 'Volume', 'OpenInt'], axis=1).T
        data.insert(0, 'Ticker', ticker)
        data.insert(1, 'Pattern', "")
        # Add the stock to the main dataframe, stocks
        stocks = pd.concat([stocks, data])
    return stocks

In [195]:
stock_data = create_stock_dataframe(15)
stock_data.head()

,Ticker,Pattern,2016-01-04 00:00:00,2016-01-05 00:00:00,2016-01-06 00:00:00,2016-01-07 00:00:00,2016-01-08 00:00:00,2016-01-11 00:00:00,2016-01-12 00:00:00,2016-01-13 00:00:00,...,2016-02-16 00:00:00,2016-02-17 00:00:00,2016-02-18 00:00:00,2016-02-19 00:00:00,2016-02-22 00:00:00,2016-02-23 00:00:00,2016-02-24 00:00:00,2016-02-25 00:00:00,2016-02-26 00:00:00,2016-02-29 00:00:00
Close,a,,39.979,39.842,40.018,38.317,37.916,37.278,37.523,36.216,...,36.433,37.210,36.540,36.786,37.365,36.521,36.826,36.971,36.933,36.698
Close,aa,,21.365,20.398,18.946,18.197,17.757,17.603,16.018,15.688,...,17.944,18.829,17.944,17.391,19.690,18.850,19.359,19.601,19.601,19.735
Close,aaap,,30.670,30.880,29.960,29.650,28.000,26.790,24.710,26.090,...,26.000,26.990,30.680,31.100,32.000,31.330,32.840,33.000,32.980,33.635
Close,aaba,,31.400,32.200,32.160,30.160,30.630,30.170,30.690,29.440,...,29.280,29.370,29.420,30.040,31.170,30.670,30.950,31.360,31.370,31.790
Close,aac,,18.520,19.460,19.010,18.200,18.110,17.460,16.900,16.240,...,21.000,22.260,21.350,21.450,22.680,23.240,23.060,22.730,21.740,20.400


In [191]:
%%capture
# Capture restricts the cell from producing output into the notebook
# For each stock, create a graph and save it
def create_charts(data):
    for i in range(0, len(data)):
        plt.figure(figsize=(40, 20))
        data.iloc[i][2:-1].interpolate().plot()
        
        plt.savefig('Images/' + stock_data['Ticker'][i] + '.png')

In [193]:
%%capture
create_charts(stock_data)